In [1]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import numpy as np

In [2]:
CHAR_SET_LEN = 10
#image height
IMAGE_HEIGHT = 60
#image width
IMAGE_WIDTH = 160
#batch size
BATCH_SIZE = 32
#tfrecord file path
TFRECOD_FILE = "captcha/train.tfrecords"

#placeholder
x = tf.placeholder(tf.float32,[None,224,224])
y0 = tf.placeholder(tf.float32,[None])
y1 = tf.placeholder(tf.float32,[None])
y2 = tf.placeholder(tf.float32,[None])
y3 = tf.placeholder(tf.float32,[None])

#define the learning rate
lr = tf.Variable(0.003, dtype=tf.float32)


def read_and_decode(filename):
    print "inniininininiiini"
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    #The read function takes the filename queue, dequeues file names and extracts the image and label
    _,serialized_example = reader.read(filename_queue)
    #print len(serialized_example)
    print type(serialized_example)
    print "aaaaaa"
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'image':tf.FixedLenFeature([],tf.string),
                                          'label0':tf.FixedLenFeature([],tf.int64),
                                          'label1':tf.FixedLenFeature([],tf.int64),
                                          'label2':tf.FixedLenFeature([],tf.int64),
                                          'label3':tf.FixedLenFeature([],tf.int64),
                                      })
    # get the image data
    image = tf.decode_raw(features['image'],tf.uint8)
    print "Eroor is here"
    image = tf.reshape(image,[224,224])
    #image preprocessing
    image = tf.cast(image,tf.float32) / 255.0
    image = tf.subtract(image,0.5)
    image = tf.multiply(image,2.0)
    print "test"
    print type(image)
    print image.shape
    print "test"
    #get the label
    label0 = tf.cast(features['label0'],tf.int32)
    label1 = tf.cast(features['label1'],tf.int32)
    label2 = tf.cast(features['label2'],tf.int32)
    label3 = tf.cast(features['label3'],tf.int32)
    
    
    return image,label0,label1,label2,label3

In [3]:
# get the image and label infomation
image,label0,label1,label2,label3 = read_and_decode(TFRECOD_FILE)


# shuffle the input
image_batch,label1_batch0,label1_batch1,label1_batch2,label1_batch3 = tf.train.shuffle_batch(
    [image,label0,label1,label2,label3],batch_size = BATCH_SIZE,
    capacity = 50000, min_after_dequeue = 10000,num_threads=1)
print type(image_batch)



# define the network structure
train_network_fn = nets_factory.get_network_fn('alexnet_v2',
                                              num_classes=CHAR_SET_LEN,
                                              weight_decay=0.0005,
                                              is_training= True)


X = tf.reshape(x,[BATCH_SIZE,224,224,1])
logits0,logits1,logits2,logits3,end_points = train_network_fn(X)
print logits0

# convert the label to one_hot
one_hot_labels0 = tf.one_hot(indices=tf.cast(y0,tf.int32),depth=CHAR_SET_LEN)
one_hot_labels1 = tf.one_hot(indices=tf.cast(y1,tf.int32),depth=CHAR_SET_LEN)
one_hot_labels2 = tf.one_hot(indices=tf.cast(y2,tf.int32),depth=CHAR_SET_LEN)
one_hot_labels3 = tf.one_hot(indices=tf.cast(y3,tf.int32),depth=CHAR_SET_LEN)

# calculate loss
loss0 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits0,labels=one_hot_labels0))
loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits1,labels=one_hot_labels1))
loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits2,labels=one_hot_labels2))
loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits3,labels=one_hot_labels3))

# calculate the total loss
total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0
#optimize the total loss
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)

# calculate the accuracy 
correct_prediction0 = tf.equal(tf.argmax(one_hot_labels0,1),tf.arg_max(logits0,1))
accuracy0 = tf.reduce_mean(tf.cast(correct_prediction0,tf.float32))

correct_prediction1 = tf.equal(tf.argmax(one_hot_labels1,1),tf.arg_max(logits1,1))
accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1,tf.float32))

correct_prediction2 = tf.equal(tf.argmax(one_hot_labels2,1),tf.arg_max(logits2,1))
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2,tf.float32))

correct_prediction3 = tf.equal(tf.argmax(one_hot_labels3,1),tf.arg_max(logits3,1))
accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3,tf.float32))

# save the model
saver = tf.train.Saver()

with tf.Session() as sess:
    
   
    #initialzation
    sess.run(tf.global_variables_initializer())
    
    # 
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(6001):
        b_image,b_label0,b_label1,b_label2,b_label3 = sess.run([image_batch,label1_batch0,label1_batch1,label1_batch2,label1_batch3])
        print b_image.shape
        sess.run(optimizer, feed_dict={x: b_image, y0:b_label0, y1: b_label1, y2 : b_label2, y3:b_label3})
        
        if i % 20 == 0:
            if i % 2000 == 0:
                sess.run(tf.assign(lr,lr/3))
            acc0,acc1,acc2,acc3,loss_=sess.run([accuracy0,accuracy1,accuracy2,accuracy3,total_loss],feed_dict={x: b_image,
                                                                                                             y0: b_label0,
                                                                                                             y1: b_label1,
                                                                                                             y2: b_label2,
                                                                                                             y3: b_label3})
            learning_rate = sess.run(lr)
            print("Iter:%d Loss:%.3f Accuracy:%.2f,%.2f,%.2f%.2f Learning_rate:%.4f" % (i,loss_,acc0,acc1,acc2,acc3,learning_rate))
            
            if i == 6000:
                saver.save(sees,"captcha/models/crack_captcha.model",global_step=i)
                break
    
    coord.request_stop()
    coord.join(threads)
    

inniininininiiini
<class 'tensorflow.python.framework.ops.Tensor'>
aaaaaa
Eroor is here
test
<class 'tensorflow.python.framework.ops.Tensor'>
(224, 224)
test
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("alexnet_v2/fc8_0/squeezed:0", shape=(32, 10), dtype=float32)
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


KeyboardInterrupt: 